In [ ]:
import sys
sys.path.append(r"C:\Users\Talha Ali\Desktop\machine learning labs\timeseires")

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
import os
from keras.callbacks import Callback

In [14]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [16]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [18]:
model1 = create_rnn()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 24, 8)               │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 20)                  │             580 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\lab 9\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Talha Ali\Desktop\machine learning labs\lab 9'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
C:\Users\PMLS\AppData\Local\Temp\ipykernel_13540\4169972906.py:3: SyntaxWarning: invalid escape sequence '\h'
  FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
C:\Users\PMLS\AppData\Local\Temp\ipykernel_13540\4169972906.py:4: SyntaxWarning: invalid escape sequence '\h'
  JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])


In [24]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [26]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [ ]:
import os
path_dataset =r'C:\Users\Talha Ali\Desktop\machine learning labs\lab7'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [30]:
time_steps=24
num_features=21

In [32]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.8372375965118408 sec


In [34]:
epochs = 30
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/30
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0781 - mae: 0.0781 - mape: 64.7811
Epoch 1: val_loss improved from inf to 0.01802, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 24s 7ms/step - loss: 0.0780 - mae: 0.0780 - mape: 65.4104 - val_loss: 0.0180 - val_mae: 0.0180 - val_mape: 8.1471
Epoch 2/30
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0147 - mae: 0.0147 - mape: 36.6136
Epoch 2: val_loss improved from 0.01802 to 0.01143, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0147 - mae: 0.0147 - mape: 36.7070 - val_loss: 0.0114 - val_mae: 0.0114 - val_mape: 5.1611
Epoch 3/30
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0124 - mae: 0.0124 - mape: 9.4913
Epoch 3: val_loss improved from 0.01143 to 0.01006, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0124 - mae: 0.0124 - mape: 9.5981 - val_loss: 0.0101 - val_mae: 0.0101 - val_mape: 4.3280
Epoch 4/30
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0116 - mae: 0.0116 - mape: 37.1193
Epoch 4: val_loss did not improve from 0.01006
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0116 - mae: 0.0116 - mape: 37.1272 - val_loss: 0.0102 - val_mae: 0.0102 - val_mape: 4.4457
Epoch 5/30
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0110 - mae: 0.0110 - mape: 12.4235
Epoch 5: val_loss did not improve from 0.01006
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0110 - mae: 0.0110 - mape: 12.4182 - val_loss: 0.0102 - val_mae: 0.0102 - val_mape: 4.4684
Epoch 6/30
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0105 - mae: 0.0105 - mape: 23.4658
Epoch 6: val_loss improved from 0.01006 to 0.00987, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0006-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0105 - mae: 0.0105 - mape: 23.4879 - val_loss: 0.0099 - val_mae: 0.0099 - val_mape: 4.2032
Epoch 7/30
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0102 - mae: 0.0102 - mape: 312.9035
Epoch 7: val_loss improved from 0.00987 to 0.00905, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0007-loss0.01.h5


...

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0089 - mae: 0.0089 - mape: 19.7987 - val_loss: 0.0081 - val_mae: 0.0081 - val_mape: 3.5730
Epoch 14/30
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0089 - mae: 0.0089 - mape: 7.5371
Epoch 14: val_loss did not improve from 0.00811
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0089 - mae: 0.0089 - mape: 7.7229 - val_loss: 0.0083 - val_mae: 0.0083 - val_mape: 4.0339
Epoch 15/30
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0087 - mae: 0.0087 - mape: 21.1332
Epoch 15: val_loss improved from 0.00811 to 0.00789, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0015-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0087 - mae: 0.0087 - mape: 21.1951 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.4843
Epoch 16/30
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0087 - mae: 0.0087 - mape: 10.1909
Epoch 16: val_loss did not improve from 0.00789
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0087 - mae: 0.0087 - mape: 10.3833 - val_loss: 0.0086 - val_mae: 0.0086 - val_mape: 3.7419
Epoch 17/30
2644/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0087 - mae: 0.0087 - mape: 19.8760
Epoch 17: val_loss improved from 0.00789 to 0.00776, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0017-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0087 - mae: 0.0087 - mape: 19.9222 - val_loss: 0.0078 - val_mae: 0.0078 - val_mape: 3.2990
Epoch 18/30
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0086 - mae: 0.0086 - mape: 158.2738
Epoch 18: val_loss did not improve from 0.00776
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0086 - mae: 0.0086 - mape: 158.1468 - val_loss: 0.0106 - val_mae: 0.0106 - val_mape: 4.4995
Epoch 19/30
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0088 - mae: 0.0088 - mape: 15.6189
Epoch 19: val_loss did not improve from 0.00776
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0088 - mae: 0.0088 - mape: 15.6988 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.6686
Epoch 20/30
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0085 - mae: 0.0085 - mape: 3.0801
Epoch 20: val_loss improved from 0.00776 to 0.00731, saving model to C:\Users\PMLS\ml\lab 9\\E1-cp-0020-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0085 - mae: 0.0085 - mape: 3.1072 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.2174
Epoch 21/30
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0085 - mae: 0.0085 - mape: 3.9516
Epoch 21: val_loss did not improve from 0.00731
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0085 - mae: 0.0085 - mape: 3.9522 - val_loss: 0.0078 - val_mae: 0.0078 - val_mape: 3.3094
Epoch 22/30
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0083 - mae: 0.0083 - mape: 6.5206
Epoch 22: val_loss did not improve from 0.00731
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0083 - mae: 0.0083 - mape: 6.5243 - val_loss: 0.0083 - val_mae: 0.0083 - val_mape: 3.6448
Epoch 23/30
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0084 - mae: 0.0084 - mape: 103.5964
Epoch 23: val_loss did not improve from 0.00731
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - loss: 0.0084 - mae: 0.0084 - mape: 103.4416 - val_loss: 0.0080 - val_mae: 0.008

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0081 - mae: 0.0081 - mape: 32.6050 - val_loss: 0.0070 - val_mae: 0.0070 - val_mape: 3.0135
Epoch 27/30
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0081 - mae: 0.0081 - mape: 14.3306
Epoch 27: val_loss did not improve from 0.00703
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0081 - mae: 0.0081 - mape: 14.3518 - val_loss: 0.0072 - val_mae: 0.0072 - val_mape: 3.1359
Epoch 28/30
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0081 - mae: 0.0081 - mape: 78.4088
Epoch 28: val_loss did not improve from 0.00703
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0081 - mae: 0.0081 - mape: 78.4721 - val_loss: 0.0096 - val_mae: 0.0096 - val_mape: 4.1811
Epoch 29/30
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0081 - mae: 0.0081 - mape: 2.9160
Epoch 29: val_loss did not improve from 0.00703
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0081 - mae: 0.0081 - mape: 2.9207 - val_loss: 0.0092 - val_mae: 0.00

In [ ]:



model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labs\lab 9\E1-cp-0026-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Mean Absolute Error (MAE): 108.01
Median Absolute Error (MedAE): 87.04
Mean Squared Error (MSE): 20360.26
Root Mean Squared Error (RMSE): 142.69
Mean Absolute Percentage Error (MAPE): 0.74 %
Median Absolute Percentage Error (MDAPE): 0.6 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\lab 9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Talha Ali\Desktop\machine learning labs\lab 9\E1-cp-0026-loss0.01.h5'
start_epoch= 54

In [ ]:
from keras.metrics import mean_absolute_error as mae
from keras.metrics import mean_absolute_percentage_error as mape
from keras.models import load_model
import numpy as np
from keras import backend as K

# Define your model path (replace with your actual path)
model_path = r'C:\Users\Talha Ali\Desktop\machine learning labs\lab9\E1-cp-0028-loss0.01.h5'

# construct the callback to save only the *best* model to disk
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1 = TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1, TrainingMonitor1]

# if there is no specific model checkpoint supplied, initialize new model
if model_path is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mean_absolute_error', 
                 optimizer=opt, 
                 metrics=["mean_absolute_error", "mean_absolute_percentage_error"])
else:
    print("[INFO] loading {}...".format(model_path))
    try:
        # First try loading with custom objects
        model = load_model(model_path, custom_objects={
            'mae': mae,
            'mape': mape,
            'mean_absolute_error': mae,
            'mean_absolute_percentage_error': mape
        })
    except Exception as e:
        print(f"First load attempt failed: {e}")
        try:
            # Fallback to loading without custom objects
            model = load_model(model_path)
        except Exception as e:
            print(f"Second load attempt failed: {e}")
            try:
                # Final fallback - load without compilation
                model = load_model(model_path, compile=False)
                model.compile(loss='mean_absolute_error',
                            optimizer=Adam(1e-4),
                            metrics=[mae, mape])
            except Exception as e:
                print(f"Final load attempt failed: {e}")
                raise RuntimeError("Could not load model with any method")

    # CORRECTED LEARNING RATE UPDATE APPROACH
    try:
        # Get the actual learning rate variable
        lr_var = model.optimizer.learning_rate
        if not isinstance(lr_var, str):  # Only proceed if it's not a string
            old_lr = float(K.get_value(lr_var))
            print("[INFO] old learning rate: {}".format(old_lr))
            
            # Set new learning rate - using Keras backend properly
            K.set_value(lr_var, 1e-4)
            
            new_lr = float(K.get_value(lr_var))
            print("[INFO] new learning rate: {}".format(new_lr))
        else:
            print("[WARNING] Learning rate appears to be a string, cannot update")
    except Exception as e:
        print(f"[WARNING] Could not update learning rate: {e}")
        # Alternative approach if the above fails
        try:
            model.optimizer.learning_rate.assign(1e-4)
            print("[INFO] Updated learning rate using direct assignment")
        except Exception as e:
            print(f"[WARNING] Failed to update learning rate completely: {e}")

[INFO] loading C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5...
[WARNING] Could not update learning rate: module 'keras.api.backend' has no attribute 'get_value'
[INFO] Updated learning rate using direct assignment


In [50]:
import tensorflow as tf
import numpy as np

# ✅ Enable eager execution (so .numpy() works if used in callbacks or debug)
tf.config.run_functions_eagerly(True)

# ✅ Ensure data is NumPy arrays
train_X = np.array(train_X)
train_y = np.array(train_y)
validation_X = np.array(validation_X)
validation_y = np.array(validation_y)

# ✅ Recreate the optimizer (IMPORTANT)
optimizer = tf.keras.optimizers.Adam()  # Or the one you prefer

# ✅ Recompile the model
model.compile(optimizer=optimizer,
              loss='mse',             # Change if you're using classification
              metrics=['mae'])       # You can add other metrics too

# ✅ Set training parameters
epochs = 10
batch_size = 32
verbose = 1

# ✅ Fit the model
History = model.fit(
    train_X,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_X, validation_y),
    callbacks=callbacks,
    verbose=verbose
)


C:\Users\PMLS\anaconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
2470/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 122ms/step - loss: 1.1968e-04 - mae: 0.0078

KeyboardInterrupt: 

In [ ]:

model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labslab9\E2-cp-0006-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 1ms/step
Mean Absolute Error (MAE): 144.75
Median Absolute Error (MedAE): 113.65
Mean Squared Error (MSE): 36189.68
Root Mean Squared Error (RMSE): 190.24
Mean Absolute Percentage Error (MAPE): 0.98 %
Median Absolute Percentage Error (MDAPE): 0.79 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
